In [1]:
# --- Step 1: Install dependencies ---
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.0 MB/s eta 0:00:00


In [2]:
import os, cv2, numpy as np, contextlib, io
from ultralytics import YOLO
import glob


os.environ["YOLO_VERBOSE"] = "False"
os.environ["KMP_WARNINGS"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

class ObjectDetector:
    def __init__(self, model_name='yolov8m.pt'):
        self.model = YOLO(model_name)

    def detect_objects(self, image_path, rows=2, cols=2, conf_threshold=0.45, imgsz=1280):
        image = cv2.imread(image_path)
        if image is None:
            return None, None

        h, w = image.shape[:2]
        tile_h, tile_w = h // rows, w // cols
        detections_total, crops = [], []

        for i in range(rows):
            for j in range(cols):
                y1, y2 = i * tile_h, (i + 1) * tile_h
                x1, x2 = j * tile_w, (j + 1) * tile_w
                crops.append(image[y1:y2, x1:x2])

        # Suppress all YOLO output
        with contextlib.redirect_stdout(io.StringIO()):
            results = self.model.predict(crops, conf=conf_threshold, imgsz=imgsz, verbose=False)

        for res in results:
            for box in res.boxes:
                detections_total.append(int(box.cls))

        num_people = detections_total.count(0)
        num_cars = sum(1 for cls in detections_total if cls in [2, 5, 7])  # car, bus, truck


        return num_people, num_cars

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Example usage:
detector = ObjectDetector()
people, vehicle = detector.detect_objects("/content/traffic.jpg")
print(f"🧍 People detected: {people}")
print(f"🚗 Vehicles detected: {vehicle}")

In [17]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("farzadnekouei/top-view-vehicle-detection-image-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/farzadnekouei/top-view-vehicle-detection-image-dataset/versions/2


In [19]:
!ls /root/.cache/kagglehub/datasets/farzadnekouei/top-view-vehicle-detection-image-dataset/versions/2/Vehicle_Detection_Image_Dataset

data.yaml	    README.roboflow.txt  sample_video.mp4  valid
README.dataset.txt  sample_image.jpg	 train


In [ ]:
from ultralytics import YOLO
base = "/root/.cache/kagglehub/datasets/farzadnekouei/top-view-vehicle-detection-image-dataset/versions/2/Vehicle_Detection_Image_Dataset"
# Load pretrained YOLOv8m model
model = YOLO("yolov8m.pt")

# Train on your dataset
model.train(
    data=base+"/data.yaml",
    epochs=40,          # can increase to 60–100 if needed
    imgsz=960,
    batch=16,
    lr0=0.001,
    pretrained=True,
    name="yolov8m_vehicle_finetuned",
    project="Vehicle_Training"
)


Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/root/.cache/kagglehub/datasets/farzadnekouei/top-view-vehicle-detection-image-dataset/versions/2/Vehicle_Detection_Image_Dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m_vehicle_finetune

In [ ]:
import os
import random
import cv2

# Assuming your ObjectDetector class is defined
detector = ObjectDetector()

# Paths
base_path = "/kaggle/input/top-view-vehicle-detection-image-dataset/Vehicle_Detection_Image_Dataset"
val_images = os.path.join(base_path, "valid/images")
val_labels = os.path.join(base_path, "valid/labels")

# Function to count vehicles from YOLO labels
def count_true_vehicles(label_path):
    if not os.path.exists(label_path):
        return 0
    with open(label_path, "r") as f:
        lines = f.readlines()
    return len(lines)

# --- Select random subset ---
all_images = [f for f in os.listdir(val_images) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# total_samples = 50
# sample_images = random.sample(all_images, total_samples)
total_samples = len(all_images)

total_true, total_pred, errors = 0, 0, []
count = 0
for img_file in all_images:
    img_path = os.path.join(val_images, img_file)
    label_path = os.path.join(val_labels, img_file.rsplit('.', 1)[0] + ".txt")

    true_count = count_true_vehicles(label_path)
    total_true += true_count

    people, cars = detector.detect_objects(img_path)
    pred_count = cars
    total_pred += pred_count
    errors.append(abs(pred_count - true_count))
    count+=1
    print(f"{count}/{total_samples} processed")

mean_abs_error = sum(errors) / len(errors)
accuracy = 1 - abs(total_pred - total_true) / max(total_true, 1)

print(f"✅ Tested on {total_samples} images")
print(f"🧾 Total True: {total_true}, Predicted: {total_pred}")
print(f"📉 Mean Absolute Error per image: {mean_abs_error:.2f}")
print(f"🎯 Overall Accuracy: {accuracy:.2f}")


1/90 processed
2/90 processed
3/90 processed
4/90 processed
5/90 processed
6/90 processed
7/90 processed
8/90 processed
9/90 processed
10/90 processed
11/90 processed
12/90 processed
13/90 processed
14/90 processed
15/90 processed
16/90 processed
17/90 processed
18/90 processed
19/90 processed
20/90 processed
21/90 processed
22/90 processed
23/90 processed
24/90 processed
25/90 processed
26/90 processed
27/90 processed
28/90 processed
29/90 processed
30/90 processed
31/90 processed
32/90 processed
33/90 processed
34/90 processed
35/90 processed
36/90 processed
37/90 processed
38/90 processed
39/90 processed
40/90 processed
41/90 processed
42/90 processed
43/90 processed
44/90 processed
45/90 processed
46/90 processed
47/90 processed
48/90 processed
49/90 processed
50/90 processed
51/90 processed
52/90 processed
53/90 processed
54/90 processed
55/90 processed
56/90 processed
57/90 processed
58/90 processed
59/90 processed
60/90 processed
61/90 processed
62/90 processed
63/90 processed
6

## Output:
```shell
✅ Tested on 90 images
🧾 Total True: 937, Predicted: 1125
📉 Mean Absolute Error per image: 5.04
🎯 Overall Accuracy: 0.80
```

# Accuracy testing for people

In [6]:
path = kagglehub.dataset_download("fmena14/crowd-counting")
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/fmena14/crowd-counting/versions/3


In [ ]:
# Your detector class
detector = ObjectDetector()

# Paths to crowd dataset (adjust accordingly)
base = path
model = ObjectDetector()
# Load dataset
images = np.load(base + "/images.npy")
labels = np.load(base + "/labels.npy")

# Ensure shapes match
print(f"Loaded {len(images)} images and {len(labels)} labels")
# Select a random subset of 100 frames
COUNT = 100
sample_indices = random.sample(range(len(images)), COUNT)
sample_images = images[sample_indices]
sample_labels = labels[sample_indices]

total_true, total_pred, errors = 0, 0, []
count = 0

for img, true_count in zip(sample_images, sample_labels):
    count += 1
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Save temporarily since detector expects a path
    tmp_path = f"/kaggle/temp_{count}.jpg"
    cv2.imwrite(tmp_path, img_bgr)

    people, cars = detector.detect_objects(tmp_path, conf_threshold=0.25)
    os.remove(tmp_path)

    total_true += int(true_count)
    total_pred += int(people)
    errors.append(abs(int(people) - int(true_count)))

    print(f"{count}/{COUNT} processed")

mean_abs_error = sum(errors) / len(errors)
accuracy = 1 - abs(total_pred - total_true) / max(total_true, 1)

print(f"✅ Tested on {COUNT} random images")
print(f"🧾 Total True: {total_true}, Predicted: {total_pred}")
print(f"📉 Mean Absolute Error per image: {mean_abs_error:.2f}")
print(f"🎯 Overall Accuracy: {accuracy:.2f}")

Loaded 2000 images and 2000 labels


/tmp/ipython-input-952515941.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  total_true += int(true_count)
/tmp/ipython-input-952515941.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  errors.append(abs(int(people) - int(true_count)))


1/100 processed
2/100 processed
3/100 processed
4/100 processed
5/100 processed
6/100 processed
7/100 processed
8/100 processed
9/100 processed
10/100 processed
11/100 processed
12/100 processed
13/100 processed
14/100 processed
15/100 processed
16/100 processed
17/100 processed
18/100 processed
19/100 processed
20/100 processed
21/100 processed
22/100 processed
23/100 processed
24/100 processed
25/100 processed
26/100 processed
27/100 processed
28/100 processed
29/100 processed
30/100 processed
31/100 processed
32/100 processed
33/100 processed
34/100 processed
35/100 processed
36/100 processed
37/100 processed
38/100 processed
39/100 processed
40/100 processed
41/100 processed
42/100 processed
43/100 processed
44/100 processed
45/100 processed
46/100 processed
47/100 processed
48/100 processed
49/100 processed
50/100 processed
51/100 processed
52/100 processed
53/100 processed
54/100 processed
55/100 processed
56/100 processed
57/100 processed
58/100 processed
59/100 processed
60/100

## Output:
```shell
✅ Tested on 100 random images
🧾 Total True: 3088, Predicted: 2504
📉 Mean Absolute Error per image: 6.12
🎯 Overall Accuracy: 0.81
```